In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time
from __future__ import print_function
from builtins import input


from sporco.admm import bpdn
### using the ADMM algorithm 
### we can also use the PGM algorithm 
from sporco import util
from sporco import plot
plot.config_notebook_plotting()

from scipy.linalg import hadamard

import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

import time 
from scipy import stats
from matplotlib import pyplot
iHiV = pd.read_pickle("~/bge-analysis-simv3/iHiV.pkl")
B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")


import warnings
warnings.filterwarnings("ignore")

In [2]:
X = pd.read_pickle("~/bge-analysis-simv3/X16.pkl")

In [3]:
from sklearn.metrics import r2_score

In [4]:
labdl = np.round(np.logspace(-5,0.8,5),5)
lam = list(labdl)
lam.append([7.0,9.0,10.0,12.0,20.0])
lamf = list(pd.DataFrame(lam)[0].explode().values)
lamf.sort()

In [5]:
n = 2**16
random_state = None
laml = lamf

In [6]:
opt = bpdn.BPDN.Options({'Verbose': False, 'MaxMainIter': 500,
                         'RelStopTol': 1e-8, 'AutoRho': {'RsdlTarget': 1.0}})

In [7]:
sst_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-16sp-FUNCTIONS/16-list1-allmetabols.pkl")


In [8]:
metabolite_list = ["butyrate","lactate","acetate","succinate"]

In [ ]:
whper = [] 
linper = [] 
for metabol in metabolite_list : 
    datab = sst_redv1[str(metabol)]
    databdf = pd.DataFrame(datab)
    datap = list(databdf.index)
    for runs in range(3):
        for cv_splits in [7,10]:
            xs = list(datap)
            random.shuffle(xs)
            data_present = np.array(xs)
            kf = KFold(n_splits=cv_splits)
            kf.get_n_splits(data_present)
            KFold(n_splits=cv_splits, random_state=random_state, shuffle=True)
            for lmda in laml:
                for train_index, test_index in kf.split(data_present):
                    X_train, X_test = data_present[train_index], data_present[test_index]
                    rinew = list(X_train)
                    rileft = list(X_test)
                    D = iHiV[rinew,:]
                    y2 = databdf.T[rinew].T.values
                    yact = databdf.T[rileft].T.values
                    s3 = y2 
                    b = bpdn.BPDN(D, s3, lmda, opt)
                    x = b.solve()
                    yrecon = np.dot(iHiV,x.ravel())
                    whper.append([runs,lmda,str(metabol),yrecon[rileft],yact,rileft,rinew,cv_splits,len(rileft),len(rinew)])
                    D = X[rinew,:]
                    y2 = databdf.T[rinew].T.values
                    s3 = y2 
                    b = bpdn.BPDN(D, s3, lmda, opt)
                    x = b.solve()
                    yrecon = np.dot(X,x.ravel())
                    linper.append([runs,lmda,str(metabol),yrecon[rileft],yact,rileft,rinew,cv_splits,len(rileft),len(rinew)])
                    #print(runs,lamf)

In [ ]:
whdf  = pd.DataFrame(whper)
lindf = pd.DataFrame(linper)
pd.to_pickle(whdf,"7n10_cv_WHmetabols.pkl")
pd.to_pickle(lindf,"7n10_cv_linmetabols.pkl")

In [ ]:
whdf  = pd.DataFrame(whper)
lindf = pd.DataFrame(linper)

In [ ]:
whdf

In [ ]:
whdf

In [2]:
import pandas as pd
bdf = pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-16sp-FUNCTIONS-v3/inferred_betas_butyrate_optimal.pkl")

In [3]:
bdf[0]

array([16.00371457, -4.35482666, -0.08964008, ...,  0.        ,
        0.        ,  0.        ])